In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import scvelo as scv
scv.settings.verbosity=1

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [9]:
sorted([file.name for file in (TEMPDIR / 'WesselsSatija2023').glob('*')])

['GSE213957_HEX293FT_CaRPool-seq.metadata.tsv',
 'GSE213957_HEX293FTstabRNA.metadata.tsv',
 'GSE213957_SpeciesMixing_CaRPool-seq.metadata.tsv',
 'GSE213957_THP1-CaRPool-seq.metadata.tsv',
 'GSE213957_THP1_Cas13d_combinatorial_pooledScreen.counts.csv',
 'GSE213957_THP1_Cas13d_pooledScreen.counts.csv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.ADT.barcodes.tsv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.ADT.feature_ref.csv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.ADT.features.tsv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.ADT.matrix.mtx',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.GDO.feature_ref.csv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.GEXGDO.barcodes.tsv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.GEXGDO.features.tsv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.GEXGDO.matrix.mtx',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.HTO.barcodes.tsv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.HTO.feature_ref.csv',
 'HEK293FT_and_SpeciesMixing_CaRPool-seq.HTO.features.tsv',
 'HEK293FT_and_SpeciesMi

In [4]:
# There are 2 main screens: THP1-CaRPool-seq and Perturb-seq (which includes HEX293_CaRPool-seq)

In [75]:
from scipy.sparse import csr_matrix
from scipy.io import mmread
def read_count_matrix(key):
    X = csr_matrix(mmread(TEMPDIR / f'WesselsSatija2023/{key}.matrix.mtx').T)
    obs = pd.read_csv(TEMPDIR / f'WesselsSatija2023/{key}.barcodes.tsv', sep='\t', index_col=0, names=['cell_barcode'])
    var = pd.read_csv(TEMPDIR / f'WesselsSatija2023/{key}.features.tsv', sep='\t', index_col=1, names=['ENSEMBL_ID', 'gene_symbol', 'feature_type'])
    return X, obs, var

In [256]:
# This is plain Perturb-seq
X, obs, var = read_count_matrix('Perturb-seq.GEXGDO')
obs.index = [x[:-2] for x in obs.index]
adata = sc.AnnData(X, obs=obs, var=var)

# Add ADT
X_ADT, obs_ADT, var_ADT = read_count_matrix('Perturb-seq.ADT')
var_ADT = var_ADT.set_index('ENSEMBL_ID')
var_ADT.index.name='ADT_id'
var_ADT.feature_type = 'Surface Protein'
ADT = pd.DataFrame(X_ADT.A, index=obs_ADT.index, columns=var_ADT.index)
diff = adata.obs.index[~adata.obs.index.isin(ADT.index)]  #  2 idxs...
for d in diff:
    ADT.loc[d]=0
adata.obsm['Surface_protein']=ADT.loc[adata.obs_names]

# Add HTO
X_HTO, obs_HTO, var_HTO = read_count_matrix('Perturb-seq.HTO')
var_HTO = var_HTO.set_index('ENSEMBL_ID')
var_HTO.index.name='HTO_id'
var_HTO.feature_type = 'Hashtag Oligo'
HTO = pd.DataFrame(X_HTO.A, index=obs_HTO.index, columns=var_HTO.index)
diff = adata.obs.index[~adata.obs.index.isin(HTO.index)]  #  2 idxs...
for d in diff:
    HTO.loc[d]=0
adata.obsm['Sample_Tags']=HTO.loc[adata.obs_names]

# Add Metadata
tab = pd.read_csv(TEMPDIR / 'WesselsSatija2023/Perturb-seq.metadata.tsv', sep='\t')
ptab = tab[tab.Experiment=='Perturb-seq']
ptab.index = [x[:-4] for x in ptab.index]
adata = adata[ptab.index].copy()
adata.obs = ptab

In [257]:
adata.obs

,Experiment,hash.ID,Pool,BelongsToPool,TargetGenes,Type,TargetGuideCombination,TargetGuideCombination.detailed
AAACCCAAGTACAGCG,Perturb-seq,HTO-26,CRISPRko.n=1,CRISPRko.n=1,CD71,Cas9-nuclease,CD71,CD71-g1
AAACCCACAGCCGGTT,Perturb-seq,HTO-26,CRISPRko.n=1,CRISPRko.n=1,NT,Cas9-nuclease,NT,NT-g29
AAACGAAAGTCTTGGT,Perturb-seq,HTO-27,CRISPRko.n=2,CRISPRko.n=2,NT,Cas9-nuclease,NT/NT,NT/NT
AAACGCTCAGCGGTCT,Perturb-seq,HTO-27,CRISPRko.n=2,CRISPRko.n=2,CD46/CD55,Cas9-nuclease,CD46/CD55,CD46/CD55
AAAGAACCATCTTCGC,Perturb-seq,HTO-28,CRISPRko.n=3,CRISPRko.n=3,NT,Cas9-nuclease,NT/NT/NT,NT/NT/NT
...,...,...,...,...,...,...,...,...
TTTGTTGAGTGCTAGG,Perturb-seq,HTO-51,CRISPRi.V2.n=1,CRISPRi.n=1,CD55,KRAB-dCas9-MeCP2,CD55,CD55-g2
TTTGTTGCAGGTATGG,Perturb-seq,HTO-54,CRISPRi.V2.n=3,CRISPRi.n=3,CD55,KRAB-dCas9-MeCP2,NT/CD55/NT,NT/CD55/NT
TTTGTTGGTCGGCACT,Perturb-seq,HTO-52,CRISPRi.V2.n=2,CRISPRi.n=2,CD55/CD71,KRAB-dCas9-MeCP2,CD55/CD71,CD55/CD71
TTTGTTGGTTCCCACT,Perturb-seq,HTO-51,CRISPRi.V2.n=1,CRISPRi.n=1,CD46,KRAB-dCas9-MeCP2,CD46,CD46-g1


In [260]:
adata.obs.TargetGenes.unique()

array(['CD71', 'NT', 'CD46/CD55', 'CD46/CD71', 'CD46', 'CD55',
       'CD55/CD71', 'CD46/CD55/CD71'], dtype=object)

In [ ]:
# Harmonize
adata.obs.rename({
    'nCount_RNA': 'ncounts', 
    'nFeature_RNA': 'ngenes', 
    'percent.mt': 'percent_mito',
    'nCount_HTO': 'ncounts_HTO',
    'nCount_ADT': 'ncounts_ADT',
    'TenX.Lane': '10X_lane',
    'CRISPR.Array': 'CRISPRcas13_Array',
    'GenePair': 'perturbation',
    'Guides': 'guides',
    'Phase': 'Cell_cycle_phase'
}, axis=1, inplace=True)
adata.obs.perturbation[adata.obs.perturbation=='NT_NT'] = 'control'
adata.obs.perturbation = [x.replace('NT_','').replace('_NT', '') for x in adata.obs.perturbation]
adata.obs['nperts'] = [p.count('_')+1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]
adata.obs['perturbation_type'] = 'CRISPR-cas13'
adata.obs['disease'] = "leukemia"
adata.obs['cancer'] = True
adata.obs['tissue_type']="cell_line"
adata.obs["cell_line"] = "THP-1"
adata.obs["celltype"] = 'monocytes'
adata.obs['organism'] = 'human'
annotate_qc(adata, species='human')
adata.obs.index.name = 'cell_barcode'
assert_annotations(adata)

In [259]:
adata.obs.Pool.unique()

array(['CRISPRko.n=1', 'CRISPRko.n=2', 'CRISPRko.n=3', 'CRISPRi.V1.n=1',
       'CRISPRi.V1.n=3', 'CRISPRi.V1.n=2', 'CRISPRi.V2.n=3',
       'CRISPRi.V2.n=1', 'CRISPRi.V2.n=2'], dtype=object)

In [210]:
adatas = []
for j in tqdm(['1', '2', '3', '4']):
    X, obs, var = read_count_matrix(f'THP1-CaRPool-seq_and_HEK293FTstabRNA.GEXGDO{j}')
    adata = sc.AnnData(X, obs=obs, var=var)
    adata.obs.index = [x[:-2] for x in obs.index]
    adata.obs.index = [f'L{j}_{x}' for x in adata.obs.index]
    
    # Add ADT
    X_ADT, obs_ADT, var_ADT = read_count_matrix(f'THP1-CaRPool-seq_and_HEK293FTstabRNA.ADT{j}')
    var_ADT = var_ADT.set_index('ENSEMBL_ID')
    var_ADT.index.name='ADT_id'
    var_ADT.feature_type = 'Surface Protein'
    obs_ADT.index = [f'L{j}_{x}' for x in obs_ADT.index]
    ADT = pd.DataFrame(X_ADT.A, index=obs_ADT.index, columns=var_ADT.index)
    diff = adata.obs.index[~adata.obs.index.isin(ADT.index)]  #  2 idxs...
    for d in diff:
        ADT.loc[d]=0
    adata.obsm['Surface_protein']=ADT.loc[adata.obs_names]

    # Add HTO
    X_HTO, obs_HTO, var_HTO = read_count_matrix(f'THP1-CaRPool-seq_and_HEK293FTstabRNA.HTO{j}')
    var_HTO = var_HTO.set_index('ENSEMBL_ID')
    var_HTO.index.name='HTO_id'
    var_HTO.feature_type = 'Hashtag Oligo'
    obs_HTO.index = [f'L{j}_{x}' for x in obs_HTO.index]
    HTO = pd.DataFrame(X_HTO.A, index=obs_HTO.index, columns=var_HTO.index)
    diff = adata.obs.index[~adata.obs.index.isin(HTO.index)]  #  2 idxs...
    for d in diff:
        HTO.loc[d]=0
    adata.obsm['Sample_Tags']=HTO.loc[adata.obs_names]
    
    adata.var_names_make_unique()  # Otherwise error
    adatas.append(adata)

  0%|          | 0/4 [00:00<?, ?it/s]

In [254]:
adata = sc.concat(adatas)
tab = pd.read_csv(TEMPDIR / 'WesselsSatija2023/GSE213957_THP1-CaRPool-seq.metadata.tsv', sep='\t')
tab.index = [x[:-2] for x in tab.index]
adata = adata[tab.index].copy()
adata.obs = tab

# Harmonize
adata.obs.drop(['S.Score', 'G2M.Score'], axis=1, inplace=True)
adata.obs.rename({
    'nCount_RNA': 'ncounts', 
    'nFeature_RNA': 'ngenes', 
    'percent.mt': 'percent_mito',
    'nCount_HTO': 'ncounts_HTO',
    'nCount_ADT': 'ncounts_ADT',
    'TenX.Lane': '10X_lane',
    'CRISPR.Array': 'CRISPRcas13_Array',
    'GenePair': 'perturbation',
    'Guides': 'guides',
    'Phase': 'Cell_cycle_phase'
}, axis=1, inplace=True)
adata.obs.perturbation[adata.obs.perturbation=='NT_NT'] = 'control'
adata.obs.perturbation = [x.replace('NT_','').replace('_NT', '') for x in adata.obs.perturbation]
adata.obs['nperts'] = [p.count('_')+1-p.count('control') if type(p)==str else 0 for p in adata.obs.perturbation]
adata.obs['perturbation_type'] = 'CRISPR-cas13'
adata.obs['disease'] = "leukemia"
adata.obs['cancer'] = True
adata.obs['tissue_type']="cell_line"
adata.obs["cell_line"] = "THP-1"
adata.obs["celltype"] = 'monocytes'
adata.obs['organism'] = 'human'
annotate_qc(adata, species='human')
adata.obs.index.name = 'cell_barcode'
assert_annotations(adata)

In [255]:
adata.obs

,ncounts,ngenes,percent_mito,Cell_cycle_phase,ncounts_HTO,HTO,ncounts_ADT,10X_lane,guides,nCount_GUIDE,...,g1.by.g2,nperts,perturbation_type,disease,cancer,tissue_type,cell_line,celltype,organism,percent_ribo
cell_barcode,,,,,,,,,,,,,,,,,,,,,
L1_TGTGAGTAGTTAACAG,15908,3744,8.536585,G1,1792,HTO-52,13376,L1,UID-046,45,...,14.5,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,26.941154
L1_TCAATTCCACTTACAG,23236,4756,6.046652,G1,2290,HTO-22,6731,L1,UID-216,55,...,12.0,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,26.565652
L1_CGTTCTGAGGTTACCT,19865,4883,13.063176,S,4218,HTO-44,12765,L1,UID-380,85,...,38.0,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,19.637129
L1_CTCATTAAGGTCCCGT,19946,4492,7.159330,G1,2640,HTO-21,9473,L1,UID-020,114,...,48.5,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,27.546850
L1_CCAAGCGAGCAAACAT,21521,4830,3.814878,G1,767,HTO-52,8282,L1,UID-232,57,...,46.0,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,24.354756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
L4_TTGGATGGTTGTTGCA,5028,1850,9.924423,G1,1465,HTO-29,3647,L4,UID-100,28,...,7.0,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,24.426424
L4_TCGGGACCAACAGAGC,1952,1062,9.631148,G1,483,HTO-22,1277,L4,UID-187,9,...,7.0,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,14.839368
L4_ACTATGGGTTGGTGTT,10274,3078,10.375706,S,571,HTO-29,4928,L4,UID-131,17,...,inf,2,CRISPR-cas13,leukemia,True,cell_line,THP-1,monocytes,human,17.654489


In [253]:

from warnings import warn
from itertools import product
from tqdm import tqdm
from scipy.io import mmwrite
from scipy.sparse import issparse
from scipy.stats import zscore
from sklearn.metrics import pairwise_distances
from scipy.cluster.hierarchy import distance, linkage, dendrogram
from shutil import copyfileobj

def assert_annotations(adata):
    assert adata.obs.index.name == 'cell_barcode', "Please set your index name correctly using 'adata.obs.index.names = ['cell_barcode']'"
    for obs_key in ['disease', 'cancer', 'tissue_type', 'perturbation', 'organism', 'perturbation_type', 'ncounts', 'ngenes', 'nperts', 'percent_mito', 'percent_ribo']:
        assert obs_key in adata.obs.keys(), f"There is no column `{obs_key}` in adata.obs. Please annotate it (worst case: set it all to None)!"
    if (adata.obs.tissue_type[0] == 'cell_line') or (adata.obs.tissue_type[0] == 'organoid'):
        assert 'cell_line' in adata.obs.keys(), f"There is no column `cell_line` in adata.obs, even though this is a cell culture model. Please annotate it (worst case: set it all to None)!"
    for var_key in ['ncounts', 'ncells']:
        assert var_key in adata.var.keys(), f"There is no column `{var_key}` in adata.var. Please annotate it (worst case: set it all to None)!"

def read_from_singles(folder, use_h5=True):
    """
    Read AnnData object from folder with single file format.

    This function reads single file format from folder. It creates an
    AnnData object like this:
    adata.X <-- counts.mtx.gz
    adata.var <-- var.csv
    adata.obs <-- obs.csv
    adata.obsm <-- obsm/
    adata.varm <-- varm/
    Note that adata.uns is not recoverable.

    Parameters
    ----------
    adata : AnnData (Annotated data matrix) object
        Data object to write.
    folder : str
        Path to folder with data in single file format.

    Returns
    -------
    AnnData
        An AnnData object reconstructed from single files in folder.
    """
    if use_h5:
        h5_files = [f for f in os.listdir(f'{folder}') if '.h5' in f]
        if len(h5_files)==1:
            return sc.read(f'{folder}/{h5_files[0]}')
        else:
            warn('Found multiple h5 files. Reading from singles instead...')

    # read from singles
    adata = sc.read_mtx(f'{folder}/counts.mtx.gz')
    adata.obs = pd.read_csv(f'{folder}/obs.csv', index_col=0)
    adata.var = pd.read_csv(f'{folder}/var.csv', index_col=0)
    if os.path.isdir(f'{folder}/obsm/'):
        for obsm_file in [f for f in os.listdir(f'{folder}/obsm/') if os.path.isfile(os.path.join(f'{folder}/obsm/', f))]:
            key = obsm_file.replace('.gz', '').replace('.csv', '')
            adata.obsm[key] = np.loadtxt(f'{folder}/obsm/{obsm_file}', delimiter=',')
    if os.path.isdir(f'{folder}/varm/'):
        for varm_file in [f for f in os.listdir(f'{folder}/varm/') if os.path.isfile(os.path.join(f'{folder}/varm/', f))]:
            key = varm_file.replace('.gz', '').replace('.csv', '')
            adata.varm[key] = np.loadtxt(f'{folder}/varm/{varm_file}', delimiter=',')
    return adata

def rsum(X, axis):
    # handles sparse sum, returns array instead of matrix object
    return np.ravel(np.sum(X, axis=axis))

def detect_organism(adata):
    if np.sum(adata.var_names.str.startswith('MT-')) > 0:
        return 'human'
    elif np.sum(adata.var_names.str.startswith('mt-')) > 0:
        return 'mouse'
    else:
        print('Could not reliably detect organism from mito genes. Setting as human.')
        return 'human'

def specify_genes(genes, source_species='mouse', target_species='human'):
    return genes

def get_genefamily_percentage(adata, key='MT-', start=True, name='mito'):
    keys = key if isinstance(key, list) else [key, '____ignore____']
    if start:
        family_genes = np.logical_or(*[adata.var_names.str.startswith(k) for k in keys])
    else:
        family_genes = np.logical_or(*[adata.var_names.str.endswith(k) for k in keys])
    if issparse(adata.X):
        adata.obs['percent_'+name] = np.sum(
            adata[:, family_genes].X, axis=1).A1 * 100 / np.sum(adata.X, axis=1).A1
    else:
        adata.obs['percent_'+name] = np.sum(
            adata[:, family_genes].X, axis=1) * 100 / np.sum(adata.X, axis=1)

def get_mito_percentage(adata, species='human'):
    key = 'MT-' if species == 'human' else 'mt-'
    get_genefamily_percentage(adata, key=key, start=True, name='mito')

def get_ribo_percentage(adata, species='human'):
    key = specify_genes(['RPS', 'RPL'], target_species=species)
    get_genefamily_percentage(adata, key=key, start=True, name='ribo')

def get_hemo_percentage(adata, species='human'):
    key = specify_genes(['HBA', 'HBB'], target_species=species)
    get_genefamily_percentage(adata, key=key, start=True, name='hemo')

def annotate_qc(adata, species='detect'):
    """
    Annotates quality controls and such to existing adata (inplace).

    This function adds (if they do not yet exist):
    adata.obs:
        - ncounts
        - ngenes
        - nperts
        - organism (inferred from `species`)
        - percent_mito
        - percent_ribo
        - percent_hemo (if tissue_type exists in obs and is `'primary'`)
    adata.obs.index:
        - updates name of index to be `'cell_barcode'`
    adata.var:
        - ncounts
        - ncells

    Parameters
    ----------
    adata : AnnData (Annotated data matrix) object
        Data object to write.
    species : `str` from `['detect', 'human', 'mouse']` (default: `'detect'`)
        Species for gene family percentage calling. If detect, heuristically
        tries to infer if data is mouse or human based on presence of
        either "MT-" or "mt-" genes.

    Returns
    -------
    None
    """

    # qc counts
    if 'ncounts' not in adata.obs.keys():
        adata.obs['ncounts'] = rsum(adata.X, axis=1)
    if 'ngenes' not in adata.obs.keys():
        adata.obs['ngenes'] = rsum(adata.X>0, axis=1)
    if 'ncounts' not in adata.var.keys():
        adata.var['ncounts'] = rsum(adata.X, axis=0)
    if 'ncells' not in adata.var.keys():
        adata.var['ncells'] = rsum(adata.X>0, axis=0)

    species = detect_organism(adata) if species == 'detect' else species
    if 'organism' not in adata.obs.keys():
        adata.obs['organism'] = species
    # gene modules
    if 'percent_mito' not in adata.obs.keys():
        get_mito_percentage(adata, species)
    if 'percent_ribo' not in adata.obs.keys():
        get_ribo_percentage(adata, species)
    if adata.obs['tissue_type'][0]=='tissue' and 'percent_hemo' not in adata.obs.keys():
        get_hemo_percentage(adata, species)

    # rename index
    adata.obs.index.names = ['cell_barcode']

    # perturbation stuff
    if 'nperts' not in adata.obs.keys():
        adata.obs['nperts'] = [len(x.split('_')) if isinstance(x, str) else 0 for x in adata.obs.perturbation]